https://archive.ax.dev/versions/0.4.1/tutorials/multiobjective_optimization.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html

https://botorch.readthedocs.io/en/latest/_modules/botorch/test_functions/multi_objective.html

In [226]:
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from botorch.test_functions.multi_objective import BraninCurrin,ZDT1,ZDT2,ZDT3
from complexity.glch_experiments_functions import save_glch_data



In [160]:
class BaseEvaluator:

    def __init__(self,name,best_init_pt,best_init_pt_img_loc,best_h1_list,best_h2_list,func):
        self.name = name
        self.best_init_pt = best_init_pt # best initial point
        self.best_init_pt_img_loc = best_init_pt_img_loc # best initial point image location
        self.best_h1_list = best_h1_list
        self.best_h2_list = best_h2_list
        self.func = func

    def evaluate_one(self,parameters):
        evaluation = self.func(torch.tensor([parameters.get("h1"), parameters.get("h2")]))
        return {"a": evaluation[0].item(), "b": evaluation[1].item()}

    def evaluate_many(self,possible_values):
        
        evaluations = []
        for h1 in possible_values["h1"]:
            for h2 in possible_values["h2"]:
                e = self.evaluate_one({"h1":h1/max(possible_values["h1"]),"h2":h2/max(possible_values["h2"])})
                evaluations.append({"h1":h1,"h2":h2,"a": e["a"],"b": e["b"]})
        
        data = pd.DataFrame(evaluations)
    
        data["topology"] = (data["h1"]).astype(int).apply(lambda x: f"{x:02d}") + "_" + \
            (data["h2"]).astype(int).apply(lambda x: f"{x:02d}")
    
        data = data.set_index("topology")
    
        return data


class BraninCurrinEvaluator(BaseEvaluator):

    def __init__(self):
        super().__init__(
            "branin_currin",
            [40,0],
            "left",
            list(np.arange(0,40 + 1))[::-1],
            list(np.arange(0,10 + 1)),
            BraninCurrin(negate=False).to(dtype=torch.double,device=torch.device("cpu"))
        )

class ZDT1Evaluator(BaseEvaluator):

    def __init__(self):
        super().__init__(
            "zdt1",
            [0,0],
            "left",
            list(np.arange(0,40 + 1)),
            list(np.arange(0,10 + 1)),
            ZDT1(2).to(dtype=torch.double,device=torch.device("cpu"))
        )

class ZDT2Evaluator(BaseEvaluator):

    def __init__(self):
        super().__init__(
            "zdt2",
            [0,0],
            "left",
            list(np.arange(0,40 + 1)),
            list(np.arange(0,10 + 1)),
            ZDT2(2).to(dtype=torch.double,device=torch.device("cpu"))
        )

class ZDT3Evaluator(BaseEvaluator):

    def __init__(self):
        super().__init__(
            "zdt3",
            [0,10],
            "left",
            list(np.arange(0,40 + 1)),
            list(np.arange(0,10 + 1))[::-1],
            ZDT3(2).to(dtype=torch.double,device=torch.device("cpu"))
        )

In [191]:

from ax.core.search_space import SearchSpace
from ax.core.parameter import ParameterType, ChoiceParameter
from complexity.ax_utils import build_optimization_config_mohpo
from complexity.ax_utils import get_hv_from_df
from ax.metrics.noisy_function import NoisyFunctionMetric

In [194]:
def hypervolume(data,ref_point):

    def params_to_label(h1,h2):
        widths = [h1,h2]
        return '_'.join(map(lambda x: f"{x:02d}",widths))
    
    def label_to_params(label):
        split_label = label.split("_")
        h1 = int(split_label[0])
        h2 = int(split_label[1])
        return {"h1":h1,"h2":h2}
    
    h1 = ChoiceParameter(
        name="h1", 
        values=list(data["h1"].drop_duplicates(keep='first')), 
        parameter_type=ParameterType.INT, 
        is_ordered=False, 
        sort_values=False
    )
    h2 = ChoiceParameter(
        name="h2", 
        values=list(data["h2"].drop_duplicates(keep='first')), 
        parameter_type=ParameterType.INT, 
        is_ordered=False, 
        sort_values=False
    )
    
    parameters=[h1, h2]
    
    class MetricA(NoisyFunctionMetric):
        def f(self, x: np.ndarray) -> float:
            return float(data.loc[params_to_label(*x),"a"])
    
    class MetricB(NoisyFunctionMetric):
        def f(self, x: np.ndarray) -> float:
            return float(data.loc[params_to_label(*x),"b"])
    
    metric_a = MetricA("a", ["h1", "h2"], noise_sd=0.0, lower_is_better=True)
    metric_b = MetricB("b", ["h1", "h2"], noise_sd=0.0, lower_is_better=True)
    
    metrics = [metric_a,metric_b]
    
    search_space = SearchSpace(parameters=parameters)
    
    optimization_config = build_optimization_config_mohpo(metrics,ref_point)
    
    hv = get_hv_from_df(search_space,optimization_config,data,label_to_params)

    return hv

# Exploration

In [96]:
evaluator = BraninCurrinEvaluator()
# evaluator = ZDT1Evaluator()
# evaluator = ZDT2Evaluator()
# evaluator = ZDT3Evaluator()

In [97]:
data = evaluator.evaluate_many({"h1": list(np.arange(0,40 + 1)),"h2": list(np.arange(0,10 + 1))})

In [243]:
# data.corr()

In [172]:
# sns.pairplot(data)

In [99]:
# data.loc["40_10"]

In [100]:
# data.loc["00_00"]

In [166]:
# data[data["b"] ==data["b"].min()]

In [167]:
# data[data["a"] ==data["a"].min()]

In [165]:
# data[data["a"] ==data["a"].max()]

In [164]:
# data[data["b"] ==data["b"].max()]

In [163]:
# import matplotlib
# plt.close('all')
# matplotlib.use("Qt5Agg")
# %matplotlib inline
# plt.scatter(list(data["a"]),list(data["b"]))

In [92]:
# plt.scatter(list(data["h1"]),list(data["h2"]))

# Experiment

In [134]:

def glch_synthetic_2d(evaluator):
    
    possible_values = {"h1": evaluator.best_h1_list,"h2": evaluator.best_h2_list} # possible values for the hyperparameters
    data = evaluator.evaluate_many(possible_values) # dataframe that maps hyperparameters to objectives
    to_str_method = lambda p : '_'.join(map(lambda x: f"{x:02d}",[p["h1"],p["h2"]])) # converts hyperparameters to dataframe index
    initial_values = {"h1":evaluator.best_init_pt[0],"h2":evaluator.best_init_pt[1]} # initial hyperparameters
    x_in_log_scale=False # used when plotting, when the x-axis varies too much, e.g., network parameters
    algo="glch" # options are glch (2d) and gho (1d)
    select_function="angle_rule" # Used when algo == glch. options are tie_break,gift_wrapping,angle_rule. Recommended: angle_rule
    constrained=True # Used when select_function == gift_wrapping/angle_rule. We only implemented the constrained tie_break select function
    fldr="glch_synthetic_data_results" # results are stored here
    debug=False # used to debug the algorithm implementation if any problem is detected
    debug_folder="glch_synthetic_data_debug" # debug results are stored here
    title = evaluator.name # identifies the experiment
    start=evaluator.best_init_pt_img_loc # if start == left, the algorithm finds the lch from left to right
    weights = [1,1] # used only in 1d problems
    axes = ["a","b"] # names of the objectives in the data variable
    axes_ranges=[None,None] # used only to adjust the plots
    axes_aliases=["1st Objective","2nd Objective"] # names of the objectives on the plots
    axes_scales = [data["a"].max() - data["a"].min(),data["b"].max() - data["b"].min()] # used only if select_function == tie_break
    
    
    save_glch_data(
        algo,
        data,possible_values,axes,initial_values,to_str_method,constrained,weights,start,
        axes_scales,
        debug,title,debug_folder,select_function,
        x_in_log_scale,axes_ranges,axes_aliases,fldr)

In [162]:
# glch_synthetic_2d(BraninCurrinEvaluator())

In [145]:
# glch_synthetic_2d(ZDT1Evaluator())

In [146]:
# glch_synthetic_2d(ZDT2Evaluator())

In [148]:
# glch_synthetic_2d(ZDT3Evaluator())

# Hypervolume

In [198]:
evaluator = BraninCurrinEvaluator()
# evaluator = ZDT1Evaluator()
# evaluator = ZDT2Evaluator()
# evaluator = ZDT3Evaluator()

In [199]:
data = evaluator.evaluate_many({"h1": evaluator.best_h1_list,"h2": evaluator.best_h2_list})

In [200]:
ref_point = data[["h1","h2"]].max().values * 1.1

In [201]:
hypervolume(data,ref_point)

/home/lucas/.virtualenvs/axenv/lib/python3.9/site-packages/ax/modelbridge/modelbridge_utils.py:1030: UserWarning: FYI: The default behavior of `get_pareto_frontier_and_configs` when `transform_outcomes_and_configs` is not specified has changed. Previously, the default was `transform_outcomes_and_configs=True`; now this argument is deprecated and behavior is as if `transform_outcomes_and_configs=False`. You did not specify `transform_outcomes_and_configs`, so this warning requires no action.
  frontier_observations, f, obj_w, obj_t = get_pareto_frontier_and_configs(


393.6653116423588

In [224]:
def read_glch_data(csv_path):
    glch_data = pd.read_csv(csv_path, header=0,index_col=0)
    glch_data = glch_data[["topology","h1","h2","a","b"]].drop_duplicates(keep="first").set_index("topology")
    return glch_data

In [217]:
glch_data = read_glch_data("/home/lucas/Documents/perceptronac/complexity/scripts/glch_experiments/glch_synthetic_data_results/" + \
f"glch2D_angle_rule_constrained_{evaluator.name}_history.csv")

In [218]:
hypervolume(glch_data,ref_point)

/home/lucas/.virtualenvs/axenv/lib/python3.9/site-packages/ax/modelbridge/modelbridge_utils.py:1030: UserWarning: FYI: The default behavior of `get_pareto_frontier_and_configs` when `transform_outcomes_and_configs` is not specified has changed. Previously, the default was `transform_outcomes_and_configs=True`; now this argument is deprecated and behavior is as if `transform_outcomes_and_configs=False`. You did not specify `transform_outcomes_and_configs`, so this warning requires no action.
  frontier_observations, f, obj_w, obj_t = get_pareto_frontier_and_configs(


379.1665667122213

In [223]:
(379.1665667122213 - 393.6653116423588)/393.6653116423588 # hypervolume improvement percentage (HVIP)

-0.0368301308277562

# Table

In [248]:
glch_results_folder = "/home/lucas/Documents/perceptronac/complexity/scripts/glch_experiments/glch_synthetic_data_results/"

SPC = 12

table = []
table.append("{}\t {}\t {}\t {}\t {}\t {}\t {}\t {}\n".format(
    "Function".ljust(SPC),
    "HVDP".ljust(SPC),
    "Visited Nets".ljust(SPC),
    "Total Nets".ljust(SPC),
    "ρ(x1,y1)".ljust(SPC),
    "ρ(x2,y1)".ljust(SPC), # Pearson correlation coefficient
    "ρ(x1,y2)".ljust(SPC),
    "ρ(x2,y2)".ljust(SPC),
))
for evaluator in [BraninCurrinEvaluator(),ZDT1Evaluator(),ZDT2Evaluator(),ZDT3Evaluator()]:

    data = evaluator.evaluate_many({"h1": evaluator.best_h1_list,"h2": evaluator.best_h2_list})
    ref_point = data[["h1","h2"]].max().values * 1.1

    n_total_networks = len(set(data.index))
    
    max_hv = hypervolume(data,ref_point)

    glch_results_file = f"glch2D_angle_rule_constrained_{evaluator.name}_history.csv"

    glch_data = read_glch_data(glch_results_folder+glch_results_file)

    hv = hypervolume(glch_data,ref_point)

    hvdp = (hv - max_hv)/max_hv

    n_visited_networks = len(set(glch_data.index))

    corr_h1_a = data.corr().loc["h1","a"]
    corr_h2_a = data.corr().loc["h2","a"]
    corr_h1_b = data.corr().loc["h1","b"]
    corr_h2_b = data.corr().loc["h2","b"]

    table.append("{}\t {}\t {}\t {}\t {}\t {}\t {}\t {}\n".format(
        str(evaluator.name).ljust(SPC),
        "{:.3f}".format(hvdp).ljust(SPC),
        str(n_visited_networks).ljust(SPC),
        str(n_total_networks).ljust(SPC),
        "{:.3f}".format(corr_h1_a).ljust(SPC),
        "{:.3f}".format(corr_h2_a).ljust(SPC),
        "{:.3f}".format(corr_h1_b).ljust(SPC),
        "{:.3f}".format(corr_h2_b).ljust(SPC),
    ))

print("".join(table))

/home/lucas/.virtualenvs/axenv/lib/python3.9/site-packages/ax/modelbridge/modelbridge_utils.py:1030: UserWarning: FYI: The default behavior of `get_pareto_frontier_and_configs` when `transform_outcomes_and_configs` is not specified has changed. Previously, the default was `transform_outcomes_and_configs=True`; now this argument is deprecated and behavior is as if `transform_outcomes_and_configs=False`. You did not specify `transform_outcomes_and_configs`, so this warning requires no action.
  frontier_observations, f, obj_w, obj_t = get_pareto_frontier_and_configs(
/home/lucas/.virtualenvs/axenv/lib/python3.9/site-packages/ax/modelbridge/modelbridge_utils.py:1030: UserWarning: FYI: The default behavior of `get_pareto_frontier_and_configs` when `transform_outcomes_and_configs` is not specified has changed. Previously, the default was `transform_outcomes_and_configs=True`; now this argument is deprecated and behavior is as if `transform_outcomes_and_configs=False`. You did not specify `t

Function    	 HVDP        	 Visited Nets	 Total Nets  	 ρ(x1,y1)    	 ρ(x2,y1)    	 ρ(x1,y2)    	 ρ(x2,y2)    
branin_currin	 -0.037      	 61          	 451         	 0.015       	 0.353       	 -0.025      	 -0.843      
zdt1        	 0.000       	 91          	 451         	 1.000       	 0.000       	 -0.219      	 0.972       
zdt2        	 0.000       	 91          	 451         	 1.000       	 0.000       	 -0.029      	 0.999       
zdt3        	 0.000       	 62          	 451         	 1.000       	 -0.000      	 -0.462      	 0.477       



/home/lucas/.virtualenvs/axenv/lib/python3.9/site-packages/ax/modelbridge/modelbridge_utils.py:1030: UserWarning: FYI: The default behavior of `get_pareto_frontier_and_configs` when `transform_outcomes_and_configs` is not specified has changed. Previously, the default was `transform_outcomes_and_configs=True`; now this argument is deprecated and behavior is as if `transform_outcomes_and_configs=False`. You did not specify `transform_outcomes_and_configs`, so this warning requires no action.
  frontier_observations, f, obj_w, obj_t = get_pareto_frontier_and_configs(
